In [24]:
from __future__ import division
import os
from os.path import expanduser
import csv
import sys
import re
import numpy as np

# Parameters
ExptName 	= 'stim1'
SubjID 		= 0
RunID  		= 1
HitRate 	= 9*(1/16) # multiples of 1/16
FARate  	= 3*(1/16) # multiples of 1/16

# Path info
homeDirectory = expanduser("~")
dataDirectory = homeDirectory + os.sep + 'Google Drive/tACS_VWM_ALPHA/data/' + ExptName + os.sep + 's' + str(SubjID) + os.sep + 'setupData/'
directoryFiles = os.listdir(dataDirectory)
fileName = ''

# load setup file
for file in directoryFiles:
    match = re.match('setup-run' + str(RunID) + '.*', file, re.M|re.I)
    if match != None:
        fileName = dataDirectory + match.group()

# Columns -> TriaID, ChangeCondID, WFCond, SubCond, HFCond
setupData = np.genfromtxt(fileName, delimiter=',')
setupData=np.delete(setupData,0,0)
setupData=np.delete(setupData,5,1)


In [25]:

# these ones we can do with setupData array
# thisExp.addData('trialID', VWMTrials[i].trialID)
# thisExp.addData('ChangeTrial', VWMTrials[i].ChangeTrial)
# thisExp.addData('nDistractors', VWMTrials[i].nDistractors)
# thisExp.addData('nTargets', VWMTrials[i].nTargets)
# thisExp.addData('Cond', VWMTrials[i].condNum)
# thisExp.addData('changeHemi', changeTargHemi)
# thisExp.addData('ChangeCond', VWMTrials[i].ChangeCond)
# thisExp.addData('leftTargs', leftTargCount)
# thisExp.addData('rightTargs', rightTargCount)
# thisExp.addData('leftDists', leftDistCount)
# thisExp.addData('rightDists', rightDistCount)

# resp we need to generate with HitRate and False Alarm
# thisExp.addData('Response',resp)

# random
# thisExp.addData('RT', testResponse.rt)


array([[   0.,    3.,    2.,    5.,    2.],
       [   1.,    1.,    6.,    1.,    6.],
       [   2.,    2.,    5.,    4.,    5.],
       ..., 
       [ 381.,    2.,    1.,    2.,    1.],
       [ 382.,    3.,    3.,    1.,    3.],
       [ 383.,    3.,    2.,    3.,    2.]])

In [19]:
len(setupData)

2309